## Import Librairies

In [ ]:
import os 
import random
import pandas as pd
import numpy as np
import ast
import datetime

import matplotlib.pyplot as plt

from functools import partial

from tqdm import tqdm 

tqdm.pandas()

In [ ]:
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import RMSprop, Adam

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print(SystemError('GPU device not found'))
print('Found GPU at: {}'.format(device_name))

Version:  2.9.2
Eager mode:  True
GPU is available
Found GPU at: /device:GPU:0


In [ ]:
!nvidia-smi

Thu Feb  9 20:19:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    50W / 400W |   1656MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set up drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
DRIVEWORKSPACE_PATH = "/content/gdrive/Shareddrives/# Zone de Code #/AIRapFR"
DATASETS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}/data/datasets"

OUTPUTS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}/ai/outputs"  #txt files
LOGS_DIR =  f"{DRIVEWORKSPACE_PATH}/ai/logs"

MODELS_DIR_PATH = f"{DRIVEWORKSPACE_PATH}/ai/models"
LSTM_MODELS_DIR_PATH = f"{MODELS_DIR_PATH}/lstm"

for p in [DRIVEWORKSPACE_PATH, DATASETS_DIR_PATH, MODELS_DIR_PATH, LSTM_MODELS_DIR_PATH, OUTPUTS_DIR_PATH]:
    if not os.path.exists(p):
        raise Exception(f"{p} not exists")

In [ ]:
import sys
sys.path.append(DRIVEWORKSPACE_PATH)

In [ ]:
from utils import create_dir

## Pre-training

on fr rappers corpus

### Data for pretraining 📁

In [ ]:
from data.load_dataset import load_dataset_with_pickle

dataset_dir_path = f"{DATASETS_DIR_PATH}/dataset-1273-keryjames-20230208"
artist_name = os.path.basename(dataset_dir_path).split("-")[-2]
sentences_train_rap, next_words_train_rap, sentences_test_rap, next_words_test_rap, set_words_rap, word_index_dict_union, len_dataset_rap = load_dataset_with_pickle(dataset_dir_path, rap_corpus=True, artist_name_focus=artist_name)

Loaded dataset with pickle from /content/gdrive/Shareddrives/# Zone de Code #/AIRapFR/data/datasets/dataset-1273-keryjames-20230208/rap_corpus__26722__tok_rmpunct__min_freq_2__sent_len_10__step_1__p_test_0.1__max_year_2015.pkl.
 > len(full_dataset): 6642588
 > len(train): 5978330 # ['sport', 'et', 'en', 'chant', '\n', 'délinquant', 'rentre', 'à', "l'", 'heure'] où 
 > len(test): 664258 # ['gradés', 'dans', 'mon', 'taf', '\n', 'jalousé', 'par', 'les', 'rappeurs', 'du'] game 
 > len(set_words): 64076 # ['likenon', 'x16', 'amoureusement', 'jonque', 'tripe', 'démontage', 'sites', 'drugstore', 'visqueux', 'ateyaba'] 
 > len(word_index_dict): 64076 # [('06', 1), ('3ash9', 2), ('bolo', 3), ('excentriques', 4), ('schtroumpfettes', 5)] 


In [ ]:
word_indices = word_index_dict_union
indices_word = {v:k for k,v in word_index_dict_union.items()}

In [ ]:
SEQUENCE_LEN = len(sentences_train_rap[0])
SEQUENCE_LEN

10

La fonction `generator` sert à générer des données pour entraîner le modèle réseau de neurones LSTM. Elle prend en entrée des listes de phrases et de mots suivants, ainsi qu'une taille de batch, et renvoie des entrées et sorties pour l'entraînement.

In [ ]:
len(word_indices), len(set_words_rap)

(64625, 64076)

In [ ]:
def data_generator(sentence_list, next_word_list, batch_size, sequence_length, words_dict):
    index = 0
    while True:
        x = np.zeros((batch_size, sequence_length, len(words_dict)), dtype=bool)
        y = np.zeros((batch_size, len(words_dict)), dtype=bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, words_dict[w]] = 1
            y[i, words_dict[next_word_list[index]]] = 1

            index = (index + 1) % len(sentence_list)
        yield x, y

partial_data_generator = partial(data_generator,sequence_length=SEQUENCE_LEN,words_dict=word_indices)

for x,y in partial_data_generator(sentences_train_rap, next_words_train_rap, batch_size=128):
    print(x.shape)
    print(y.shape)
    break

(128, 10, 64625)
(128, 64625)


In [ ]:
# Data generator for fit and evaluate
def generator_embedding(sentence_list, next_words_list, batch_size, sequence_length, words_dict):
    index = 0
    while True:
        x = np.zeros((batch_size, sequence_length), dtype=np.bool)
        y = np.zeros((batch_size), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = words_dict[w]
            y[i] = next_words_list[index % len(sentence_list)]
            index = index + 1
        yield x, y
        

### Building the LSTM Model

In [ ]:
from tensorflow.keras.layers import Reshape

### Building the GRU Model

In [ ]:
def build_GRU(max_length, len_vocab, dropout=0.2,optimiser_name="rmsprop",optimizer_lr=0.01):
    """
    Build the GRU network structure and compile the model
    """
    model = Sequential(name=f"model_gru1__{max_length}_{len_vocab}_{dropout}_{optimiser_name}_{optimizer_lr}")
    model.add(GRU(128, input_shape=(max_length, len_vocab)))
    model.add(Dropout(dropout))
    model.add(Reshape((1, 128)))
    model.add(GRU(128))
    model.add(Dropout(dropout))

    model.add(Dense(len_vocab))
    model.add(Activation('softmax'))
    optimizer=Adam(learning_rate=optimizer_lr) if optimiser_name == "adam" else RMSprop(learning_rate=optimizer_lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

def build_GRU2(max_length, len_vocab, dropout=0.2):
    """
    Build the GRU network structure and compile the model
    """
    model = Sequential(name=f"model_gru2__{max_length}_{len_vocab}_{dropout}_adam")
    model.add(GRU(32, input_shape=(max_length, len_vocab)))
    model.add(Dropout(dropout))
    #model.add(Reshape((1, 128)))
    model.add(Flatten())
    model.add(Dense(len_vocab+50, activation = 'softmax'))
    model.add(Dropout(dropout))
    model.add(Dense(dropout, activation = 'softmax'))
    model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["accuracy"])
    return model


def build_EMB_GRU(max_length, len_vocab, dropout=0.2):
    """
    Build the GRU network structure and compile the model
    """
    model = Sequential(name=f"model_emb_gru__{max_length}_{len_vocab}_{dropout}_adam")

    model.add(Embedding(len_vocab, 64))
    model.add(Dropout(dropout))
    #model.add(Reshape((1, 128)))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(len_vocab))
    model.add(Activation('softmax'))
    model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model_gru_1 = build_GRU(SEQUENCE_LEN, len_vocab=len(word_indices))
model_gru_1.summary()

Model: "model_gru1__10_64625_0.2_rmsprop_0.01"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, 128)               24865920  
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 reshape_3 (Reshape)         (None, 1, 128)            0         
                                                                 
 gru_11 (GRU)                (None, 128)               99072     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64625)             8336625   
                                                                 
 activation_5 (Activation)   

In [ ]:
model_gru_2 = build_GRU2(SEQUENCE_LEN, len_vocab=len(word_indices))
model_gru_2.summary()

Model: "model_gru2__10_64625_0.2_adam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_12 (GRU)                (None, 32)                6207264   
                                                                 
 dropout_16 (Dropout)        (None, 32)                0         
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 64675)             2134275   
                                                                 
 dropout_17 (Dropout)        (None, 64675)             0         
                                                                 
 dense_11 (Dense)            (None, 0)                 0         
                                                                 
Total params: 8,341,539
Trainable par

In [ ]:
model_emb_gru = build_EMB_GRU(SEQUENCE_LEN, len_vocab=len(word_indices))
model_emb_gru.summary()

Model: "model_emb_gru__10_64625_0.2_adam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          4136000   
                                                                 
 dropout_18 (Dropout)        (None, None, 64)          0         
                                                                 
 gru_13 (GRU)                (None, 64)                24960     
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 64625)             4200625   
                                                                 
 activation_6 (Activation)   (None, 64625)             0         
                                                                 
Total params: 8,361,585
Trainable 

### Training functions and params ⚙

pre training params

In [ ]:
2**8

256

In [ ]:
BATCH_SIZE = 2**10
EPOCHS = 50
RESET_PRETRAINING = False

len_dataset_rap//BATCH_SIZE, 2**9

(6486, 512)

fine tuning params

In [ ]:
BATCH_SIZE_FINE = 256
EPOCHS_FINE = 10
RESET_FINE_TRAINING = False

train fct

In [ ]:
def train_model_bis(model,epochs,batch_size,len_sentences_dataset,sentences_train, next_words_train,sentences_test, next_words_test,callbacks_list = None ):
    #todo : n_teps
    n_steps = len_sentences_dataset//batch_size + 1
    #n_steps = 2**9
    
    history = model.fit(partial_data_generator(sentences_train, next_words_train, batch_size=batch_size),
                              steps_per_epoch=n_steps,
                              epochs=epochs, callbacks=callbacks_list,

                              validation_data=partial_data_generator(sentences_test, next_words_test, batch_size=batch_size), 
                              validation_steps=n_steps )
    return history, model
    
train_model = partial(train_model_bis,len_sentences_dataset=len_dataset_rap)

In [ ]:
def train_model_bis2(model,epochs,batch_size,len_sentences_dataset,sentences_train, next_words_train,sentences_test, next_words_test,callbacks_list = None ):
    #todo : n_teps
    n_steps = len_sentences_dataset//batch_size + 1
    #n_steps = 2**9
    
    history = model.fit(partial_data_generator(sentences_train, next_words_train, batch_size=batch_size),
                              steps_per_epoch=n_steps,
                              epochs=epochs, callbacks=callbacks_list,

                              validation_data=partial_data_generator(sentences_test, next_words_test, batch_size=batch_size), 
                              validation_steps=n_steps )
    return history, model
    
train_model2 = partial(train_model_bis,len_sentences_dataset=len_dataset_rap)

### Generate Text & Tools

In [ ]:
SEQUENCE_LEN

10

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.where(preds == 0, 1e-8, preds) # remplacer toutes les valeurs égales à zéro par une petite valeur proche de zéro
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) #todo : prendre le deuxième
    index_max_1 = np.argmax(probas) 
    
    # Trier les prédictions pour obtenir les indices des deux plus grands éléments
    sorted_indices = np.argsort(preds)[::-1][:2]
    
    return sorted_indices[0], sorted_indices[1]


def generate_text(model, starting_words,window_size, words_indices, indices_words, max_length=50, diversity=1):
    assert max_length > len(starting_words),f"max_length {max_length} must > len(starting_words) {len(starting_words)}"
    length = max_length - len(starting_words)
    
    generated = starting_words[:]
    sentence = starting_words[-window_size:]
    for i in range(length):
        x_pred = np.zeros((1, window_size, len(words_indices)))
        #print(sentence)
        for t, word in enumerate(sentence):
            x_pred[0, t, words_indices[word]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        
        next_index, else_next_index = sample(preds, diversity)
        #print([indices_words[next_index],indices_words[else_next_index]])
        next_word = indices_words[else_next_index if sentence[-1] == "\n" and indices_word[next_index] == "\n"  else next_index]
        #print(generated)
        generated.append(next_word)
        sentence = sentence[-(window_size-1):] + [next_word]
    return generated

#generate_text(model_lstm_1, ['je',"suis"],window_size=SEQUENCE_LEN, words_indices=word_indices, indices_words=indices_word, max_length=25,diversity=.1)

In [ ]:
def get_str_time_now():
    return datetime.datetime.now().strftime("%Y%m%d%H%M")

In [ ]:
def on_epoch_end(epoch, logs, model, sequence_len, words_indices, indices_words,start_time_str, max_length=15,epochs_looked = [0,25,50]):
    if epochs_looked is None:
        return
    if epoch in epochs_looked:
        first_words = ['je',"suis"]
        output_filepath = os.path.join(OUTPUTS_DIR_PATH,f"outputs__{model.name}__{start_time_str}.txt")
        

        with open(output_filepath, 'w' if not os.path.exists(output_filepath) else 'a', encoding='utf8') as output_file:
            h1 = f'\n----- Generating text after Epoch: {epoch} ({" ".join(first_words)})'
            print(h1)
            output_file.write(h1)
            for diversity in [0.2, 0.5, 1.0]:
                h2 = f'----- Diversity: {diversity} ----- '
                print(h2)
                output_file.write(h2)
                generated_text_list = generate_text(model, first_words ,window_size=sequence_len, words_indices=word_indices, indices_words=indices_word, diversity=diversity, max_length=max_length)
                generated_text = ' '.join(generated_text_list)
                print(generated_text)
                output_file.write(generated_text)

In [ ]:
def plot_learning_curves(history, title=""):
    acc      = history.history["accuracy"]
    loss     = history.history["loss"]
    val_acc  = history.history["val_accuracy"]
    val_loss = history.history["val_loss"]
    epochs = range(len(acc))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    
    fig.suptitle(title, fontsize="x-large")
    
    ax1.plot(epochs, acc, label="Entraînement")
    ax1.plot(epochs, val_acc, label="Validation")
    ax1.set_title("Accuracy - Données entraînement vs. validation.")
    ax1.set_ylabel("Accuracy (%)")
    ax1.set_xlabel("Epoch")
    ax1.legend()
    
    ax2.plot(epochs, loss, label="Entraînement")
    ax2.plot(epochs, val_loss, label="Validation")
    ax2.set_title("Perte - Données entraînement vs. validation.")
    ax2.set_ylabel('Perte')
    ax2.set_xlabel('Epoch')
    ax2.legend()

    fig.show()

### TensorBoard 📊

In [ ]:
%tensorboard --logdir "$LOGS_DIR"

### Pre-training 🔧 (model_gru_1)

on fr rappers corpus

#### checkpoint

In [ ]:
weights_file = create_dir(LSTM_MODELS_DIR_PATH ,"pre-trained") + f"/{model_gru_1.name}.hdf5"
checkpoint = ModelCheckpoint(weights_file, monitor='val_loss',  save_best_only=True, save_weights_only=False) 

if os.path.exists(weights_file) and not RESET_PRETRAINING:
    model_gru_1.load_weights(weights_file)
    print("Chargé les poids précédents")
else:
    print("Pas de poids précédents trouvés, entraînement à partir de zéro")
                                               

Pas de poids précédents trouvés, entraînement à partir de zéro


#### early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

#### print on epoch end

In [ ]:

print_callback_gru_1 = LambdaCallback(on_epoch_end=partial(on_epoch_end, model=model_gru_1, sequence_len=SEQUENCE_LEN, words_indices=word_indices, indices_words=indices_word,start_time_str=get_str_time_now(), max_length=15,epochs_looked = None)
)  


#### tensorboard 

In [ ]:
logdir = os.path.join(LOGS_DIR, datetime.datetime.now().strftime(f"{model_gru_1.name} pre-trained %Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#### 🔥 Launch pre-training 🦾

In [ ]:
history_gru_1, model_gru_1 = train_model(model=model_gru_1,epochs=EPOCHS,batch_size=BATCH_SIZE, sentences_train=sentences_train_rap, next_words_train=next_words_train_rap,sentences_test=sentences_test_rap, next_words_test=next_words_test_rap,callbacks_list=[print_callback_gru_1, checkpoint, early_stopping, tensorboard_callback])

Epoch 1/50
3351/6487 [==============>...............] - ETA: 15:22 - loss: 5.8252 - accuracy: 0.1786

In [ ]:
plot_learning_curves(history_gru_1, title=f"Modèle GRU 1 ({model_gru_1.name})")

### ... free space 🗑️

In [ ]:
del model_gru_1

### Pre-training 🔧 (model_gru_2)

on fr rappers corpus

#### checkpoint

In [ ]:
weights_file = create_dir(LSTM_MODELS_DIR_PATH ,"pre-trained") + f"/{model_gru_2.name}.hdf5"
checkpoint = ModelCheckpoint(weights_file, monitor='val_loss',  save_best_only=True, save_weights_only=False) 

if os.path.exists(weights_file) and not RESET_PRETRAINING:
    model_gru_2.load_weights(weights_file)
    print("Chargé les poids précédents")
else:
    print("Pas de poids précédents trouvés, entraînement à partir de zéro")
                                               

Pas de poids précédents trouvés, entraînement à partir de zéro


#### early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

#### print on epoch end

In [ ]:

print_callback_gru_2 = LambdaCallback(on_epoch_end=partial(on_epoch_end, model=model_gru_2, sequence_len=SEQUENCE_LEN, words_indices=word_indices, indices_words=indices_word,start_time_str=get_str_time_now(), max_length=15,epochs_looked = None)
)  


#### tensorboard 

In [ ]:
logdir = os.path.join(LOGS_DIR, datetime.datetime.now().strftime(f"{model_gru_2.name} pre-trained %Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#### 🔥 Launch pre-training 🦾

In [ ]:
history_gru_2, model_gru_2 = train_model(model=model_gru_2,epochs=EPOCHS,batch_size=BATCH_SIZE, sentences_train=sentences_train_rap, next_words_train=next_words_train_rap,sentences_test=sentences_test_rap, next_words_test=next_words_test_rap,callbacks_list=[print_callback_gru_2, checkpoint, early_stopping, tensorboard_callback])

Epoch 1/50
 126/6487 [..............................] - ETA: 30:34 - loss: 6.9683 - accuracy: 0.0992

In [ ]:
plot_learning_curves(history_gru_1, title=f"Modèle GRU 2 ({model_gru_1.name})")

### ... free space 🗑️

In [ ]:
del model_gru_2

### Pre-training 🔧 (model_emb_gru)

on fr rappers corpus

#### checkpoint

In [ ]:
weights_file = create_dir(LSTM_MODELS_DIR_PATH ,"pre-trained") + f"/{model_emb_gru.name}.hdf5"
checkpoint = ModelCheckpoint(weights_file, monitor='val_loss',  save_best_only=True, save_weights_only=False) 

if os.path.exists(weights_file) and not RESET_PRETRAINING:
    model_emb_gru.load_weights(weights_file)
    print("Chargé les poids précédents")
else:
    print("Pas de poids précédents trouvés, entraînement à partir de zéro")
                                               

Pas de poids précédents trouvés, entraînement à partir de zéro


#### early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

#### print on epoch end

In [ ]:

print_callback_emb_gru = LambdaCallback(on_epoch_end=partial(on_epoch_end, model=model_emb_gru, sequence_len=SEQUENCE_LEN, words_indices=word_indices, indices_words=indices_word,start_time_str=get_str_time_now(), max_length=15,epochs_looked = None)
)  


#### tensorboard 

In [ ]:
logdir = os.path.join(LOGS_DIR, datetime.datetime.now().strftime(f"{model_emb_gru.name} pre-trained %Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#### 🔥 Launch pre-training 🦾

In [ ]:
history_gru_2, model_emb_gru = train_model(model=model_emb_gru,epochs=EPOCHS,batch_size=BATCH_SIZE, sentences_train=sentences_train_rap, next_words_train=next_words_train_rap,sentences_test=sentences_test_rap, next_words_test=next_words_test_rap,callbacks_list=[print_callback_emb_gru, checkpoint, early_stopping, tensorboard_callback])

Epoch 1/50
 126/6487 [..............................] - ETA: 30:34 - loss: 6.9683 - accuracy: 0.0992

In [ ]:
plot_learning_curves(history_gru_1, title=f"Modèle EMB GRU ({model_emb_gru.name})")

### ... free space 🗑️

In [ ]:
del model_emb_gru

## Fine-tuning
 on 1 artist corpus 

##### Data for finetuning 📁

In [ ]:
sentences_train_artist, next_words_train_artist, sentences_test_artist, next_words_test_artist, set_words_artist, word_index_dict_union, len_dataset = load_dataset_with_pickle(dataset_dir_path, rap_corpus=False, artist_name_focus=artist_name)

##### load pre-trained

In [ ]:
# Charger les poids du modèle précédemment entraîné
weights_file = LSTM_MODELS_DIR_PATH + "/pre-trained/model_lstm_1.hdf5"
#del model_lstm_1
model_pretrained_lstm_1 = load_model(weights_file)
print(model_pretrained_lstm_1)
for diversity in [0.2,0.5,0.8,1.0]:
    print(f'----- Diversity: {diversity}')
    generated_text = generate_text(model_pretrained_lstm_1, ['je',"suis"],window_size=SEQUENCE_LEN, words_indices=word_indices, indices_words=indices_word, diversity=diversity, max_length=100)
    
    print(' '.join(generated_text))

##### checkpoint final model

In [ ]:
weights_file = LSTM_MODELS_DIR_PATH + f"/model_lstm_1_{artist_name}.hdf5"
checkpoint = ModelCheckpoint(weights_file, monitor='val_loss',  save_best_only=True, save_weights_only=False) 

if os.path.exists(weights_file) and not RESET_FINE_TRAINING:
    model_lstm_1.load_weights(weights_file)
    print("Chargé les poids précédents")
else:
    print("Pas de poids précédents trouvés, entraînement à partir de zéro")
                                               

##### early stopping final model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

##### tensorboard

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("model_lstm_1 final %Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#### 🔥 Launch training 🦾

In [ ]:
# Compiler à nouveau le modèle
model_pretrained_lstm_1.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle sur les données de l'artiste particulier
history_final_lstm_1,model_final_lstm_1 = train_model(model=model_lstm_1,epochs=EPOCHS_FINE,batch_size=BATCH_SIZE_FINE, sentences_train=sentences_train_artist, next_words_train=next_words_train_artist,
                             sentences_test=sentences_test_artist, next_words_test=next_words_test_artist, callbacks_list=[print_callback_lstm_1,checkpoint,early_stopping,tensorboard_callback ])

In [ ]:
plot_learning_curves(history_final_lstm_1, title="Modèle Final LSTM")

##### save final 💾

In [ ]:
weights_file = LSTM_MODELS_DIR_PATH + "/" + f"{model_final_lstm_1.name}.hdf5"
model_final_lstm_1.save(weights_file)